# Stable Diffusion Web UI Colab

A browser interface based on Gradio library for Stable Diffusion running on Google Colab.
For more detailed info please visit [this](https://en.wikipedia.org/wiki/Stable_Diffusion).

<img src="https://raw.githubusercontent.com/buhuipao/sd-webui-colab/master/assets/sd-example.png" width="100%">

If you want to add models, just add model files to correct sub-directory of `/content/gdrive/MyDrive/stable-diffusion-webui`, maybe it in your google driver.

Github: [buhuipao/sd-webui-colab](https://github.com/buhuipao/sd-webui-colab).

This colab is based on:
* [camenduru/stable-diffusion-webui-colab](https://github.com/camenduru/stable-diffusion-webui-colab)
* [TheLastBen/fast-stable-diffusion](https://github.com/TheLastBen/fast-stable-diffusion)
* [SlavyanDesu/sd-webui-colab](https://github.com/SlavyanDesu/sd-webui-colab)


In [ ]:
#@title ## Install Stable Diffusion
from IPython.display import clear_output
from google.colab import drive
import os
import time
import shutil

# Variables
model_path = "/content/gdrive/MyDrive/stable-diffusion-webui/models/Stable-diffusion"
ext_path = "/content/gdrive/MyDrive/stable-diffusion-webui/extensions"
vae_path = "/content/gdrive/MyDrive/stable-diffusion-webui/models/VAE"
lora_path = "/content/gdrive/MyDrive/stable-diffusion-webui/models/Lora"
controlnet_path = "/content/gdrive/MyDrive/stable-diffusion-webui/extensions/sd-webui-controlnet/models"
embd_path = "/content/gdrive/MyDrive/stable-diffusion-webui/embeddings"

#@markdown <small><font color=gray> **Note**: Check `Use_Gdrive` to use your Google Drive storage.</small><br>
Use_Gdrive = True #@param {type:"boolean"}

if Use_Gdrive == True:
  drive.mount('/content/gdrive')

if not os.path.exists("/content/gdrive/MyDrive/"):
  print("Google drive not connected, using Colab storage instead...")
  !mkdir -p /content/gdrive/MyDrive/

# Cloning Repo
%cd /content/gdrive/MyDrive
!git clone --depth 1 -q --branch master https://github.com/AUTOMATIC1111/stable-diffusion-webui
%cd /content/gdrive/MyDrive/stable-diffusion-webui
!git reset --hard
!rm webui.sh
!git pull
#clear_output()

# Installing Requirements
!apt -y update -qq
!wget http://launchpadlibrarian.net/367274644/libgoogle-perftools-dev_2.5-2.2ubuntu3_amd64.deb
!wget http://launchpadlibrarian.net/367274642/google-perftools_2.5-2.2ubuntu3_all.deb
!wget http://launchpadlibrarian.net/367274648/libtcmalloc-minimal4_2.5-2.2ubuntu3_amd64.deb
!wget http://launchpadlibrarian.net/367274647/libgoogle-perftools4_2.5-2.2ubuntu3_amd64.deb
!apt install -qq libunwind8-dev
!apt install libcairo2-dev pkg-config python3-dev 
!dpkg -i *.deb
%env LD_PRELOAD=libtcmalloc.so
!rm *.deb

# Temporary bug fix for https://github.com/AUTOMATIC1111/stable-diffusion-webui/issues/9341
%pip install huggingface-hub==0.13.4
%pip install torch==1.13.1+cu117 torchvision==0.14.1+cu117 torchtext==0.14.1 torchaudio==0.13.1 torchdata==0.5.1 --extra-index-url https://download.pytorch.org/whl/cu117
%pip install image-reward
%pip install --upgrade fastapi==0.90.1
%pip install -U xformers
#clear_output()

# Model Selection
#@markdown <small><font color=gray> **Note**: Select model that you want to use.</small><br>
Model_Name = "SD v1.5" #@param ["SD v1.5", "ChilloutMix", "Grapefruit v4.1", "Counterfeit v2.5"]

if Model_Name == "SD v1.5":
  if os.path.exists(f"{model_path}/v1-5-pruned-emaonly.ckpt"):
    print("Model already exist! Using", Model_Name)
  else:
    time.sleep(1)
    %cd $model_path
    !wget https://huggingface.co/runwayml/stable-diffusion-v1-5/resolve/main/v1-5-pruned-emaonly.ckpt -O "$model_path/v1-5-pruned-emaonly.ckpt"
elif Model_Name == "ChilloutMix":
  if os.path.exists(f"{model_path}/chilloutmix_NiPrunedFp32Fix.safetensors"):
    print("Model already exist! Using", Model_Name)
  else:
    time.sleep(1)
    %cd $model_path
    !wget https://civitai.com/api/download/models/11745 -O "$model_path/chilloutmix_NiPrunedFp32Fix.safetensors"
elif Model_Name == "Grapefruit v4.1":
  if os.path.exists(f"{model_path}/grapefruitHentaiModel_grapefruitv41.safetensors"):
    print("Model already exist! Using", Model_Name)
  else:
    time.sleep(1)
    %cd $model_path
    !wget https://civitai.com/api/download/models/29179 -O "$model_path/grapefruitHentaiModel_grapefruitv41.safetensors"
elif Model_Name == "Counterfeit v2.5":
  if os.path.exists(f"{model_path}/CounterfeitV25_25.safetensors"):
    print("Model already exist! Using", Model_Name)
  else:
    time.sleep(1)
    %cd $model_path
    !wget https://civitai.com/api/download/models/7425 -O "$model_path/CounterfeitV25_25.safetensors"
#clear_output()


# Extension
!git clone https://github.com/SlavyanDesu/sd-webui-tunnels.git $ext_path/sd-webui-tunnels
!git clone https://github.com/DominikDoom/a1111-sd-webui-tagcomplete.git $ext_path/a1111-sd-webui-tagcomplete
!git clone https://github.com/Coyote-A/ultimate-upscale-for-automatic1111.git $ext_path/ultimate-upscale-for-automatic1111
!git clone https://github.com/AlUlkesh/stable-diffusion-webui-images-browser.git $ext_path/stable-diffusion-webui-images-browser
!git clone https://github.com/camenduru/sd-civitai-browser.git $ext_path/sd-civitai-browser
!git clone https://github.com/catppuccin/stable-diffusion-webui.git $ext_path/catppuccin-theme
!git clone https://github.com/Mikubill/sd-webui-controlnet.git $ext_path/sd-webui-controlnet
!git clone https://github.com/fkunn1326/openpose-editor.git $ext_path/openpose-editor
!git clone https://github.com/KohakuBlueleaf/a1111-sd-webui-locon.git $ext_path/a1111-sd-webui-locon
!git clone https://github.com/camenduru/stable-diffusion-webui-huggingface.git $ext_path/stable-diffusion-webui-huggingface
!git clone https://github.com/kohya-ss/sd-webui-additional-networks.git $ext_path/sd-webui-additional-networks
!git clone https://github.com/butaixianran/Stable-Diffusion-Webui-Civitai-Helper $ext_path/Stable-Diffusion-Webui-Civitai-Helper
#clear_output()

# ControlNet
#@markdown <small><font color=gray> **Note**: Check `ControlNet` to download ControlNet models.</small><br>
ControlNet = True #@param {type:"boolean"}

if ControlNet == True:
  if os.path.exists(f"{controlnet_path}"):
    print("ControlNet already exist!")
  else:
    time.sleep(1)
    !wget https://huggingface.co/webui/ControlNet-modules-safetensors/resolve/main/control_canny-fp16.safetensors -P $controlnet_path
    !wget https://huggingface.co/webui/ControlNet-modules-safetensors/resolve/main/control_depth-fp16.safetensors -P $controlnet_path
    !wget https://huggingface.co/webui/ControlNet-modules-safetensors/resolve/main/control_hed-fp16.safetensors -P $controlnet_path
    !wget https://huggingface.co/webui/ControlNet-modules-safetensors/resolve/main/control_mlsd-fp16.safetensors -P $controlnet_path
    !wget https://huggingface.co/webui/ControlNet-modules-safetensors/resolve/main/control_normal-fp16.safetensors -P $controlnet_path
    !wget https://huggingface.co/webui/ControlNet-modules-safetensors/resolve/main/control_openpose-fp16.safetensors -P $controlnet_path
    !wget https://huggingface.co/webui/ControlNet-modules-safetensors/resolve/main/control_scribble-fp16.safetensors -P $controlnet_path
    !wget https://huggingface.co/webui/ControlNet-modules-safetensors/resolve/main/control_seg-fp16.safetensors -P $controlnet_path
#clear_output()

# VAE
#@markdown <small><font color=gray> **Note**: Check `VAE` to download VAE(s).</small><br>
VAE = True #@param {type:"boolean"}

if VAE == True:
  time.sleep(1)
  !wget https://huggingface.co/hakurei/waifu-diffusion-v1-4/resolve/main/vae/kl-f8-anime.ckpt -P $vae_path
  !wget https://huggingface.co/hakurei/waifu-diffusion-v1-4/resolve/main/vae/kl-f8-anime2.ckpt -P $vae_path
  !wget https://huggingface.co/gsdf/Counterfeit-V2.5/resolve/main/Counterfeit-V2.5.vae.pt -P $vae_path
  !wget https://huggingface.co/WarriorMama777/OrangeMixs/resolve/main/VAEs/orangemix.vae.pt -P $vae_path
  !wget https://huggingface.co/stabilityai/sd-vae-ft-mse-original/resolve/main/vae-ft-mse-840000-ema-pruned.ckpt -P $vae_path
#clear_output()

shutil.rmtree(embd_path)
%cd /content/gdrive/MyDrive/stable-diffusion-webui
!git clone https://huggingface.co/SlavyanDesu/embeddings

!echo 'All done!'

In [ ]:
#@title ## Load And Run Existing Stable Diffusion
#@markdown Run this instead if you have Stable Diffusion installed before.
from google.colab import drive
from IPython.display import clear_output

#@markdown <small><font color=gray> **Note**: Check `In_Gdrive` to use your Google Drive storage.</small><br>
In_Gdrive = True #@param {type:"boolean"}

if In_Gdrive == True:
  drive.mount('/content/gdrive')

%cd /content/gdrive/MyDrive/stable-diffusion-webui
!git reset --hard
!rm webui.sh
!git pull

# Installing Requirements
!apt -y update -qq
!wget http://launchpadlibrarian.net/367274644/libgoogle-perftools-dev_2.5-2.2ubuntu3_amd64.deb
!wget http://launchpadlibrarian.net/367274642/google-perftools_2.5-2.2ubuntu3_all.deb
!wget http://launchpadlibrarian.net/367274648/libtcmalloc-minimal4_2.5-2.2ubuntu3_amd64.deb
!wget http://launchpadlibrarian.net/367274647/libgoogle-perftools4_2.5-2.2ubuntu3_amd64.deb
!apt install -qq libunwind8-dev
!dpkg -i *.deb
%env LD_PRELOAD=libtcmalloc.so
!rm *.deb

# Temporary bug fix for https://github.com/AUTOMATIC1111/stable-diffusion-webui/issues/9341
%pip install huggingface-hub==0.13.4
%pip install torch==1.13.1+cu117 torchvision==0.14.1+cu117 torchtext==0.14.1 torchaudio==0.13.1 torchdata==0.5.1 --extra-index-url https://download.pytorch.org/whl/cu117
%pip install image-reward
%pip install --upgrade fastapi==0.90.1
%pip install -U xformers
#clear_output()

#@markdown Set ui username and password
UserName = "admin" #@param {type:"string"}
Password = "123456" #@param {type:"string"}

# launch_args
launch_args = "--share --xformers --enable-insecure-extension-access --reinstall-xformers --disable-safe-unpickle --remotemoe --no-half-vae --gradio-auth " + UserName + ":" + Password

# Run Stable Diffusion
!sed -i -e 's/checkout {commithash}/checkout --force {commithash}/g' /content/gdrive/MyDrive/stable-diffusion-webui/launch.py
# Fix CODEFORMER_COMMIT_HASH
!CODEFORMER_COMMIT_HASH='4724c90b6b9d5183da383f2bdae6ddf9b0bf045d' COMMANDLINE_ARGS="$launch_args" REQS_FILE="requirements.txt" python launch.py

In [ ]:
#@title ## Just Start Stable Diffusion 
from google.colab import drive
from IPython.display import clear_output

#@markdown <small><font color=gray> **Note**: Check `In_Gdrive` to use your Google Drive storage.</small><br>
In_Gdrive = True #@param {type:"boolean"}

if In_Gdrive == True:
  drive.mount('/content/gdrive')

%cd /content/gdrive/MyDrive/stable-diffusion-webui

#@markdown Set ui username and password
UserName = "admin" #@param {type:"string"}
Password = "123456" #@param {type:"string"}

# launch_args
launch_args = "--share --xformers --enable-insecure-extension-access --reinstall-xformers --disable-safe-unpickle --remotemoe --no-half-vae --gradio-auth " + UserName + ":" + Password

# Run Stable Diffusion
!sed -i -e 's/checkout {commithash}/checkout --force {commithash}/g' /content/gdrive/MyDrive/stable-diffusion-webui/launch.py
# Fix CODEFORMER_COMMIT_HASH
!CODEFORMER_COMMIT_HASH='4724c90b6b9d5183da383f2bdae6ddf9b0bf045d' COMMANDLINE_ARGS="$launch_args" REQS_FILE="requirements.txt" python launch.py

In [ ]:
#@title ## Open terminal TTY for debug
from google.colab import drive
from IPython.display import clear_output

#@markdown <small><font color=gray> **Note**: Check `In_Gdrive` to use your Google Drive storage.</small><br>
In_Gdrive = True #@param {type:"boolean"}

if In_Gdrive == True:
  drive.mount('/content/gdrive')

%cd /content/gdrive/MyDrive/stable-diffusion-webui


# Run shell
!pip install colab-xterm
%load_ext colabxterm
%xterm